### This is a WinBUGS Codes for the artificial example in Chapter 9,  Section 9.7.

#### Model: Two-level nonlinear structural equation model

Data Set Name:  YO.dat

Sample Size: N = 1555

Note: pi[g, j] is for omega(2g, j),  lb and lw are for lambda2 and lambda_1,  respectively.

In [2]:
source(".Rprofile")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract


Loading required package: coda

Loading required package: boot

This is cmdstanr version 0.6.1

- CmdStanR documentation and vignettes: mc-stan.org/cmdstanr

- CmdStan path: /home/pal_bjartan/.cmdstan/cmdstan-2.33.1

- CmdStan version: 2.33.1

This is bayesplot version 1.10.0

- Online documentation and 

#### Model

##### Check model

In [ ]:
model  <-  glue("{getwd()}/Chapter9/ch9-Stan-model.stan")
cmdstan_model(model, compile = FALSE)$check_syntax(pedantic = TRUE)

##### Compile model

In [ ]:
model <- cmdstan_model(model, 
    force_recompile = TRUE
)

#### Data


In [3]:
data <- list(
    N = c(28, 27, 25, 28, 33, 26, 18, 26, 17, 24, 26, 24, 24, 30, 23, 24, 29, 
        27, 34, 18, 20, 14, 27, 28, 28, 26, 43, 32, 43, 43, 41, 47, 45, 41, 25, 
        36, 32, 36, 44, 36, 32, 37, 36, 27, 38, 34, 39, 40, 37, 37),
    kk = c(0, 28, 55, 80, 108, 141, 167, 185, 211, 228, 252, 278, 302, 326, 
        356, 379, 403, 432, 459, 493, 511, 531, 545, 572, 600, 628, 654, 697, 
        729, 772, 815, 856, 903, 948, 989, 1014, 1050, 1082, 1118, 1162, 1198, 
        1230, 1267, 1303, 1330, 1368, 1402, 1441, 1481, 1518),
    R0 = structure(
        .Data = c(1.940, 0.775, 0.775, 0.600),
        .Dim = c(2, 2)
    ), 
    R1 = structure(
        .Data = c(13.6, -0.61, 0.48, -0.61, 0.24, 0.06, 0.48, 0.06, 0.22),
        .Dim = c(3, 3)
    ), 
    y = read.csv(
            "./Chapter9/ch9-WinBUGS-data.dat",
            header = FALSE,
            skip = 2
            ) %>%
        .[,1:(ncol(.) - 1)] %>%
        as.matrix() %>%
        unname()
)

In [4]:
data$N %>% length()

[1] 50

#### Three different initial values

In [ ]:
inits <- function() {
    list(
        lb = c(0.6, 0.6, 0.5, 2.2, 0.6, 0.4),
        lw = c(0.3, 0.3, 0.3, 0.3, 0.3, 0.3),
        mu = c(3.0, 3.5, 3.3, 1.0, 2.0, 2.2, 0.2, 0.0, 0.2), 
        psi = c(0.3,  0.3,  0.3,  0.3,  0.3,  0.3, 0.3, 0.3, 0.3),
        psd = 0.6,
        gam = c(0.2, 1.0, -0.4), 
        phip = structure(
            .Data = c(0.7, -0.1, 0.0, -0.1, 0.2, 0.0, 0.0, 0.0, 0.18),
            .Dim = c(3, 3)
        ), 
        phi = structure(
            .Data = c(0.7,  0.4, 0.4, 0.7),
            .Dim = c(2, 2)
        )
    ),

    list(
        lb = c(0.8, 0.8, 0.7, 2.5, 0.8, 0.6),
        lw = c(0.7, 0.7, 0.7, 0.7, 0.7, 0.7),
        mu = c(4.0, 4.0, 4.0, 2.0, 3.0, 3.0, 0.5, 0.4, 0.6),
        psi = c(0.5,  0.5,  0.5,  0.5,  0.5,  0.5, 0.5, 0.5, 0.5),
        psd = 0.36,
        gam = c(0.5, 1.7, 0.6),
        phip = structure(
            .Data = c(0.5, 0.1, -0.1, 0.1, 0.2, 0.0, -0.1, 0.0, 0.5),
            .Dim = c(3, 3)
        ), 
        phi = structure(
            .Data = c(0.5,  0.1, 0.1, 0.5),
            .Dim = c(2, 2)
        )
    ),

    list(
        lb = c(1.0, 1.0, 1.0, 3.0, 1.0, 1.0),
        lw = c(1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
        mu = c(4.8, 4.8, 4.8, 3.5, 4.0, 4.2, 0.8, 0.8, 0.8),
        psi = c(0.8,  0.8,  0.8,  0.8,  0.8,  0.8, 0.8, 0.8, 0.8),
        psd = 0.9,
        gam = c(0.8, 1.2, 0.0), 
        phip = structure(
            .Data = c(0.6, -0.2, 0.2, -0.2, 0.4, 0.1, 0.2, 0.1, 0.3),
            .Dim = c(3, 3)
        ), 
        phi = structure(
            .Data = c(0.9,  0.0, 0.0, 0.6),
            .Dim = c(2, 2)
        )
    )
}

In [17]:
param <- c("lb", "lw", "psi", "gam", "phi", "mu", "xi")
n.iter <- 5000
n.burnin  <- 2000

In [12]:
model.out <- R2WinBUGS::bugs(
    data, 
    inits, 
    param[-length(param)], 
    model.file = model,
    n.chains = 2,
    n.iter = n.iter,
    n.burnin = n.burnin,
    debug = TRUE,
    codaPkg = TRUE,
    bugs.directory = paste0(Sys.getenv("HOME"), "/.wine/drive_c/Program Files (x86)/WinBUGS14/"),
    program = "WinBUGS",
    working.directory = paste0(getwd(), "/Chapter9/bugs-output"),
    WINE = "/usr/bin/wine",
    WINEPATH = "/usr/bin/winepath"
)


ERROR: Error in bugs.run(n.burnin, bugs.directory, WINE = WINE, useWINE = useWINE, : Look at the log file and
try again with 'debug=TRUE' to figure out what went wrong within Bugs.


In [20]:
model.out <- R2OpenBUGS::bugs(
    data, 
    inits, 
    param[-length(param)],  
    n.iter,
    model.file = model.path,
    n.chains = 2,
    n.burnin = n.burnin,
    # debug = TRUE,
    codaPkg = TRUE,
    # OpenBUGS.pgm = paste0(Sys.getenv("HOME"), "/.wine/drive_c/Program Files (x86)/OpenBUGS/OpenBUGS323/OpenBUGS.exe"),
    working.directory = glue("{getwd()}/Chapter9/bugs-output")# ,
    # useWINE = TRUE,
    # WINE = "/usr/bin/wine",
    # WINEPATH = "/usr/bin/winepath"
)


ERROR: Error in bugs.run(n.burnin, OpenBUGS.pgm, debug = debug, WINE = WINE, : Look at the log file in  /home/pal_bjartan/Backup/PhD/SEM-test-model/Lee2007/Chapter9/bugs-output  and
try again with 'debug=TRUE' to figure out what went wrong within OpenBUGS.


In [22]:
rjags::jags.model(
    model.path,
    data,
    inits,
    n.chains = 2,
    n.adapt = n.burnin
)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Deleting model



ERROR: Error in rjags::jags.model(glue("{getwd()}/Chapter9/ch9-BUGS-model.txt"), : RUNTIME ERROR:
Unable to resolve the following parameters:
mu[4] (line 15)
Either supply values for these nodes with the data
or define them on the left hand side of a relation.


